In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import json
import os
import tqdm
import sklearn
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import log_loss, accuracy_score,f1_score
from transformers import BertModel, BertTokenizer
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

In [4]:
path = 'news'

In [6]:
train = pd.read_csv(os.path.join(path, 'train_data.csv'))
test = pd.read_csv(os.path.join(path, 'test_data.csv'))

In [8]:
topic_dict = pd.read_csv(os.path.join(path,'topic_dict.csv'))

In [27]:
def text_cleaning(sentence):
    sentence = re.sub('[^a-zA-Zㄱ-ㅣ가-힣0-9]', ' ', sentence)
    sentence = sentence.lower()
    return sentence

In [28]:
train.head()

,index,title,topic_idx,clean_text
0,0,인천→핀란드 항공기 결항…휴가철 여행객 분통,4,인천 핀란드 항공기 결항 휴가철 여행객 분통
1,1,실리콘밸리 넘어서겠다…구글 15조원 들여 美전역 거점화,4,실리콘밸리 넘어서겠다 구글 15조원 들여 전역 거점화
2,2,이란 외무 긴장완화 해결책은 미국이 경제전쟁 멈추는 것,4,이란 외무 긴장완화 해결책은 미국이 경제전쟁 멈추는 것
3,3,NYT 클린턴 측근韓기업 특수관계 조명…공과 사 맞물려종합,4,NYT 클린턴 측근 기업 특수관계 조명 공과 사 맞물려종합
4,4,시진핑 트럼프에 중미 무역협상 조속 타결 희망,4,시진핑 트럼프에 중미 무역협상 조속 타결 희망


In [29]:
train['clean_text'] = train['title'].apply(text_cleaning)

In [30]:
train.head()

,index,title,topic_idx,clean_text
0,0,인천→핀란드 항공기 결항…휴가철 여행객 분통,4,인천 핀란드 항공기 결항 휴가철 여행객 분통
1,1,실리콘밸리 넘어서겠다…구글 15조원 들여 美전역 거점화,4,실리콘밸리 넘어서겠다 구글 15조원 들여 전역 거점화
2,2,이란 외무 긴장완화 해결책은 미국이 경제전쟁 멈추는 것,4,이란 외무 긴장완화 해결책은 미국이 경제전쟁 멈추는 것
3,3,NYT 클린턴 측근韓기업 특수관계 조명…공과 사 맞물려종합,4,nyt 클린턴 측근 기업 특수관계 조명 공과 사 맞물려종합
4,4,시진핑 트럼프에 중미 무역협상 조속 타결 희망,4,시진핑 트럼프에 중미 무역협상 조속 타결 희망


In [34]:
def delete_stop(sentence):
    stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']
    for stopword in stopwords:
        pattern = stopword
        sentence = re.sub(pattern, '', sentence)
    return sentence
    

In [35]:
train['clean_text2'] = train['clean_text'].apply(delete_stop)

In [36]:
train.head()

,index,title,topic_idx,clean_text,clean_text2
0,0,인천→핀란드 항공기 결항…휴가철 여행객 분통,4,인천 핀란드 항공기 결항 휴가철 여행객 분통,인천 핀란드 항공기 결항 휴철 여행객 분통
1,1,실리콘밸리 넘어서겠다…구글 15조원 들여 美전역 거점화,4,실리콘밸리 넘어서겠다 구글 15조원 들여 전역 거점화,실리콘밸리 넘어서겠다 구글 15조원 여 전역 거점화
2,2,이란 외무 긴장완화 해결책은 미국이 경제전쟁 멈추는 것,4,이란 외무 긴장완화 해결책은 미국이 경제전쟁 멈추는 것,란 외무 긴장완화 해결책 미국 경제전쟁 멈추 것
3,3,NYT 클린턴 측근韓기업 특수관계 조명…공과 사 맞물려종합,4,nyt 클린턴 측근 기업 특수관계 조명 공과 사 맞물려종합,nyt 클린턴 측근 기업 특수관계 조명 공 사 맞물려종합
4,4,시진핑 트럼프에 중미 무역협상 조속 타결 희망,4,시진핑 트럼프에 중미 무역협상 조속 타결 희망,시진핑 트럼프 중미 무역협상 조속 타결 희망


In [37]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

In [40]:
train['token'] = train['clean_text'].apply(tokenizer.tokenize)

In [42]:
train['length'] = train['clean_text'].apply(len)

In [45]:
train['length'].describe()

count    45654.000000
mean        27.330552
std          4.968172
min          4.000000
25%         25.000000
50%         28.000000
75%         31.000000
max         44.000000
Name: length, dtype: float64

In [41]:
train.head()

,index,title,topic_idx,clean_text,clean_text2,token
0,0,인천→핀란드 항공기 결항…휴가철 여행객 분통,4,인천 핀란드 항공기 결항 휴가철 여행객 분통,인천 핀란드 항공기 결항 휴철 여행객 분통,"[인, ##천, 핀, ##란드, 항, ##공, ##기, 결, ##항, 휴, ##가,..."
1,1,실리콘밸리 넘어서겠다…구글 15조원 들여 美전역 거점화,4,실리콘밸리 넘어서겠다 구글 15조원 들여 전역 거점화,실리콘밸리 넘어서겠다 구글 15조원 여 전역 거점화,"[실, ##리, ##콘, ##밸, ##리, 넘, ##어, ##서, ##겠, ##다,..."
2,2,이란 외무 긴장완화 해결책은 미국이 경제전쟁 멈추는 것,4,이란 외무 긴장완화 해결책은 미국이 경제전쟁 멈추는 것,란 외무 긴장완화 해결책 미국 경제전쟁 멈추 것,"[이란, 외, ##무, 긴, ##장, ##완, ##화, 해, ##결, ##책, ##..."
3,3,NYT 클린턴 측근韓기업 특수관계 조명…공과 사 맞물려종합,4,nyt 클린턴 측근 기업 특수관계 조명 공과 사 맞물려종합,nyt 클린턴 측근 기업 특수관계 조명 공 사 맞물려종합,"[nyt, 클, ##린, ##턴, 측, ##근, 기, ##업, 특, ##수, ##관..."
4,4,시진핑 트럼프에 중미 무역협상 조속 타결 희망,4,시진핑 트럼프에 중미 무역협상 조속 타결 희망,시진핑 트럼프 중미 무역협상 조속 타결 희망,"[시, ##진, ##핑, 트, ##럼, ##프, ##에, 중, ##미, 무, ##역..."


In [48]:
class Bertnet(nn.Module):
    def __init__(self):
        super().__init__()
        self.bert_model = BertModel.from_pretrained('bert-base-multilingual-cased')
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
        self.max_len = 32
        self.fc = nn.Linear(self.bert_model.config.hidden_size, 512)
        self.bn = nn.BatchNorm1d(512)
        self._init_params()
        
    def _init_params(self):
        nn.init.xavier_normal_(self.fc.weight)
        nn.init.constant_(self.fc.bias, 0)
        nn.init.constant_(self.bn.weight, 1)
        nn.init.constant_(self.bn.bias, 0)
        
    def forward(self, x):
        tokenizer_output = self.tokenizer(x, truncation=True, padding=True, max_length=self.max_len)
        input_ids = torch.LongTensor(tokenizer_output['input_ids']).to('cuda')
        token_type_ids = torch.LongTensor(tokenizer_output['token_type_ids']).to('cuda')
        attention_mask = torch.LongTensor(tokenizer_output['attention_mask']).to('cuda')
        x = self.bert_model(input_ids=input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask)
        x = torch.sum(x.last_hidden_state * attention_mask.unsqueeze(-1), dim=1) / attention_mask.sum(dim=1, keepdims=True)
        x = self.fc(x)
        x = self.bn(x)
        return x
        
    

In [49]:
from sklearn.model_selection import KFold

In [50]:
train.columns

Index(['index', 'title', 'topic_idx', 'clean_text', 'clean_text2', 'token',
       'length'],
      dtype='object')

In [52]:
class NewsDataset(Dataset):

    def __init__(self, df):
        self.df = df

    def __getitem__(self, index):
        row = self.df.iloc[index]

        if 'topic_idx' in row.keys():
            target = torch.tensor(row['topic_idx'], dtype=torch.long)
            return row['title'], target
        else:
            return row['title']

    def __len__(self):
        return len(self.df)

In [79]:
def split_dataset(df):
    split_df = {}
    kfold = KFold(n_splits=5)
    for fold, (train_idx, valid_idx) in enumerate(kfold.split(df)):
        train = df.iloc[train_idx]
        valid = df.iloc[valid_idx]
        train = train.reset_index()
        valid = valid.reset_index()
        split_df[fold] = [train, valid]
    return split_df

In [80]:
data_dict = split_dataset(train)

In [101]:
for fold in range(5):
    train, valid = data_dict[fold]
    trainset = NewsDataset(train)
    validset = NewsDataset(valid)
    train_X, train_y = trainset[:][0], trainset[:][1]
    valid_X, valid_y = validset[:][0], validset[:][1]
#     train_X, train_y = train['clean_text'], train['topic_idx']
#     valid_X, valid_y = valid['clean_text'], valid['clean_text']